In [1]:
# Import Python package
import BinhoSupernova
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.definitions import *

## Getting started

#### 1. List all the Supernova devices connected to the PC host

The ``BinhoSupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [2]:
BinhoSupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#6&1062e9ec&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': 'EA3C0867C440055DABC2364357C7798',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [3]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [4]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:
    
    if supernova_message != None:

        # Print mesasage
        print(">> New message from SUPERNOVA:")
        print(supernova_message)
    
    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[0, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 6. Test communication with a basic command such as ``GET USB STRING``

The ``GET USB STRING`` command allows you to retrieve various string values related to the product, such as the manufacturer, product name, serial number, firmware version, and hardware version. This command serves as a basic test to verify the communication and functionality with the Supernova device. By successfully receiving the expected string values, you can ensure that the communication between the PC host and the Supernova device is functioning as expected.

In [8]:
# GET USB STRING - Get Manufacturer
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.MANUFACTURER)

# GET USB STRING  - Get Product Name
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.PRODUCT_NAME)

# GET USB STRING  - Get Serial Number
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.SERIAL_NUMBER)

# GET USB STRING  - Firmware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

# GET USB STRING  - Hardware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.HW_VERSION)

>> New message from SUPERNOVA:
{'id': 1, 'command': 96, 'name': 'GET USB STRING', 'length': 12, 'message': 'MN-Binho LLC'}
>> New message from SUPERNOVA:
{'id': 2, 'command': 96, 'name': 'GET USB STRING', 'length': 18, 'message': 'PR-Binho Supernova'}
>> New message from SUPERNOVA:
{'id': 3, 'command': 96, 'name': 'GET USB STRING', 'length': 35, 'message': 'SN-EA3C0867C440055DABC2364357C77981'}
>> New message from SUPERNOVA:
{'id': 4, 'command': 96, 'name': 'GET USB STRING', 'length': 19, 'message': 'FW-2.1.0-23-f73245e'}
>> New message from SUPERNOVA:
{'id': 5, 'command': 96, 'name': 'GET USB STRING', 'length': 4, 'message': 'HW-C'}


#### 7. Set the voltage of the GPIO pins

For the Rev. B Supernova device use the ``setI3cBusVoltage`` function to set the voltage of GPIO pin 1 and 2, the rest of the GPIOs work on 3.3V.


In [9]:
request_result = supernova.setI3cBusVoltage(getId(), 1800)

>> New message from SUPERNOVA:
{'id': 6, 'command': 97, 'name': 'SET I3C BUS VOLTAGE', 'result': 0}


For the Rev. C Supernova device use the ``setI2cSpiUartBusVoltage`` function to set the voltage of all the GPIO pins.

In [10]:
request_result = supernova.setI2cSpiUartBusVoltage(getId(), 1800)

>> New message from SUPERNOVA:
{'id': 7, 'command': 100, 'name': 'SET I2C-SPI-UART BUS VOLTAGE', 'result': 0}


## GPIO API

To perform the GPIO test:

**Connection Setup:**
   - Physically connect the GPIO 5 pin to the GPIO 6 pin on your device.

**1. Configure GPIO:**

When configuring a GPIO pin, consider the following configurable parameters:

* _Pin Number:_  
   - Defines the GPIO pin to configure.  

* _functionality:_ 
   - Specifies the desired functionality to be configured.  

For this case we are configuring GPIO 6 as a Digital Output:

In [11]:
request_result = supernova.gpioConfigurePin(getId(), pinNumber = GpioPinNumber.GPIO_6, functionality = GpioFunctionality.DIGITAL_OUTPUT)


>> New message from SUPERNOVA:
{'id': 8, 'command': 80, 'name': 'CONFIGURE GPIO PIN', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


And GPIO 5 as a Digital Input:

In [12]:
request_result = supernova.gpioConfigurePin(getId(), pinNumber = GpioPinNumber.GPIO_5, functionality = GpioFunctionality.DIGITAL_INPUT)


>> New message from SUPERNOVA:
{'id': 9, 'command': 80, 'name': 'CONFIGURE GPIO PIN', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


**GPIO Digital Write:**

To perform a Digital Write for a Digital Output pin we need:

* _Pin Number:_  
   - Defines the GPIO pin to write to.  

* _Logic Level:_ 
   - Specifies the desired logic level to be written.  

Since the GPIO 6 was configured as a Digital Output, we can write any logic level we desire.

In [13]:
request_result = supernova.gpioDigitalWrite(getId(), pinNumber = GpioPinNumber.GPIO_6, logicLevel = GpioLogicLevel.HIGH)

>> New message from SUPERNOVA:
{'id': 10, 'command': 81, 'name': 'GPIO DIGITAL WRITE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


**GPIO Digital Read:**

To perform a Digital Read for a Digital Input pin we need:

* _Pin Number:_  
   - Defines the GPIO pin to read from.  

This function will return the read logic level by the specified GPIO pin.

Since the GPIO 5 was configured as a Digital Input, we can read the pin logic level. Also, the GPIO_5 and GPIO_6 are connected, the read value must be the configured before.

In [14]:
request_result = supernova.gpioDigitalRead(getId(), pinNumber = GpioPinNumber.GPIO_5)

>> New message from SUPERNOVA:
{'id': 11, 'command': 82, 'name': 'GPIO DIGITAL READ', 'logic_level': 'HIGH', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


**GPIO Set Interrupts:**

GPIO pins configured as a Digital Input can have interruptions attached, to configure the interruption we need:
To perform a Digital Read for a Digital Input pin we need:

* _Pin Number:_  
   - Defines the GPIO pin to set an interruption to.  

* _Trigger:_  
   - Defines the condition to match for an interruption to be triggered.  

Since the GPIO 5 was configured as a Digital Input, we can set an interruption to it. In this case the trigger used is for both rising and falling edges, so both of them will trigger a notification back. 

In [15]:
request_result = supernova.gpioSetInterrupt(getId(), pinNumber = GpioPinNumber.GPIO_5, trigger= GpioTriggerType.TRIGGER_BOTH_EDGES)

>> New message from SUPERNOVA:
{'id': 12, 'command': 84, 'name': 'GPIO SET INTERRUPT', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


To test this, we can change the value of the GPIO 6 pin, to generate rising and falling edges. A notification from the Supernova will appear after each change in logic level.

In [16]:
request_result = supernova.gpioDigitalWrite(getId(), pinNumber = GpioPinNumber.GPIO_6, logicLevel = GpioLogicLevel.LOW)

>> New message from SUPERNOVA:
{'id': 13, 'command': 81, 'name': 'GPIO DIGITAL WRITE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 86, 'name': 'GPIO INTERRUPTION', 'pin_number': 'GPIO_5', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


In [17]:
request_result = supernova.gpioDigitalWrite(getId(), pinNumber = GpioPinNumber.GPIO_6, logicLevel = GpioLogicLevel.HIGH)

>> New message from SUPERNOVA:
{'id': 14, 'command': 81, 'name': 'GPIO DIGITAL WRITE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 86, 'name': 'GPIO INTERRUPTION', 'pin_number': 'GPIO_5', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


**GPIO Disable Interrupts:**

To stop the interruptions from a specified GPIO pin we need:

* _Pin Number:_  
   - Defines the GPIO pin to disable interrupts.   

In [18]:
request_result = supernova.gpioDisableInterrupt(getId(), pinNumber = GpioPinNumber.GPIO_5)

>> New message from SUPERNOVA:
{'id': 15, 'command': 85, 'name': 'GPIO DISABLE INTERRUPT', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


After disabling the interruptions, a change of logic level will not generate a notification.

In [19]:
request_result = supernova.gpioDigitalWrite(getId(), pinNumber = GpioPinNumber.GPIO_6, logicLevel = GpioLogicLevel.LOW)

>> New message from SUPERNOVA:
{'id': 16, 'command': 81, 'name': 'GPIO DIGITAL WRITE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


In [20]:
request_result = supernova.gpioDigitalWrite(getId(), pinNumber = GpioPinNumber.GPIO_6, logicLevel = GpioLogicLevel.HIGH)

>> New message from SUPERNOVA:
{'id': 17, 'command': 81, 'name': 'GPIO DIGITAL WRITE', 'usb_error': 'CMD_SUCCESSFUL', 'manager_error': 'GPIO_NO_ERROR', 'driver_error': 'GPIO_DRIVER_NO_ERROR'}


## Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [21]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}